In [ ]:
import folium
import graphviz as gv
import numpy as np
import pandas as pd
import heapq as hq
import math

In [ ]:
df = pd.read_csv('Lima-intersecciones.csv')

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.head()

In [ ]:
subset_of_df = df.sample(n=500)

In [ ]:
some_map = folium.Map(location=[subset_of_df['Y'].mean(),subset_of_df['X'].mean()],zoom_start=10)

In [ ]:
for row in subset_of_df.itertuples():
  some_map.add_child(folium.Marker(location=[row.Y,row.X], popup=row.ID))

In [ ]:
some_map

In [ ]:
import sys
from heapq import heapify , heappush, heappop

In [ ]:
def adjlShow():
  grafo = gv.Digraph("grafod3")
  n = len(df)
  for i in range(1000):
    #grafo.edge(str(G[i][0], str(G[i][1])))
    grafo.edge(str(df.ORIGEN[i]),str(df.DESTINO[i]), str(df.DISTANCIA[i]))
  return grafo
  

In [ ]:
def dijkstra(G, s):
  n = len(G)
  visited = [False]*n
  path = [None]*n
  cost = [math.inf]*n
  cost[s] = 0
  queue = [(0, s)]
  while queue:
    g_u, u = hq.heappop(queue)
    if not visited[u]:
      visited[u] = True
      for v, w in G[u]:
        f = g_u + w
        if f < cost[v]:
          cost[v] = f
          path[v] = u
          hq.heappush(queue, (f, v))

  return path, cost

In [ ]:
adjlShow()

In [ ]:
import math

class Vertice:
	def __init__(self, i):
		self.id = i
		self.vecinos = []
		self.visitado = False
		self.padre = None
		self.costo = float('inf')
		self.x = float('inf')
		self.y = float('inf')

	def agregarVecino(self, v, p, x, y):
		#print(v)
		#if v not in self.vecinos:
		self.vecinos.append([v, p, x, y])

class Grafica:
	def __init__(self):
		self.vertices = {}

	def agregarVertice(self, id):
		if id not in self.vertices:
			self.vertices[id] = Vertice(id)

	def agregarArista(self, a, b, p, x, y):
		if a in self.vertices and b in self.vertices:
			self.vertices[a].agregarVecino(b, p, x, y)
			#self.vertices[b].agregarVecino(a, p)

	def imprimirGrafica(self):
		for v in self.vertices:
			print("El costo del vértice "+str(self.vertices[v].id)+" es "+ str(self.vertices[v].costo)+" llegando desde "+str(self.vertices[v].padre))
			
	
	def camino(self, a, b):
		camino = []
		camino_x = []
		camino_y = []
		actual = b
		llegada = self.vertices[a].padre
		x = b
		y = b
		while actual != llegada:
			camino.insert(0, actual)
			x = self.vertices[actual].x
			camino_x.insert(0, x)
			y = self.vertices[actual].y
			camino_y.insert(0, y)
			actual = self.vertices[actual].padre
		return [camino, self.vertices[b].costo, camino_x, camino_y]

	def minimo(self, l):
		if len(l) > 0:
			m = self.vertices[l[0]].costo
			v = l[0]
			for e in l:
				if m > self.vertices[e].costo:
					m = self.vertices[e].costo
					v = e
			return v
		return None

	def dijkstra(self, a):
		if a in self.vertices:
			# 1 y 2
			self.vertices[a].costo = 0
			actual = a
			noVisitados = []
			
			for v in self.vertices:
				if v != a:
					self.vertices[v].costo = float('inf')
					#self.vertices[v].x = float('inf')
					#self.vertices[v].y = float('inf')
				self.vertices[v].padre = None
				noVisitados.append(v)

			while len(noVisitados) > 0:
				#3
				for vec in self.vertices[actual].vecinos:
					if self.vertices[vec[0]].visitado == False:
						# 3.a
						if self.vertices[actual].costo + vec[1] < self.vertices[vec[0]].costo:
							#print(vec[2])
							self.vertices[vec[0]].costo = self.vertices[actual].costo + vec[1]
							self.vertices[vec[0]].padre = actual
							self.vertices[actual].x = vec[2]
							self.vertices[actual].y = vec[3]
							#print(self.vertices[vec[0]].x)

				# 4
				self.vertices[actual].visitado = True
				noVisitados.remove(actual)

				# 5 y 6
				actual = self.minimo(noVisitados)
		else:
			return False

g = Grafica()
df = pd.read_csv('Lima-intersecciones.csv')
for i in range(10000):
  g.agregarVertice(df.ORIGEN[i])
for i in range(10000):
  g.agregarArista(df.ORIGEN[i], df.DESTINO[i], df.DISTANCIA[i], df.X[i], df.Y[i])

g.dijkstra(1)

In [ ]:
camino = g.camino(6,20393)
print(camino)
original_camino = camino[0]
peso = camino[1]
for i in range(len(original_camino)):
	print(camino[2][i])
	print(camino[3][i])


In [ ]:
#subset_of_result = original_camino.sample(n=500)}
some_map = folium.Map(location=[camino[3][0].mean(),camino[2][0].mean()],zoom_start=100)

numeros=[1,2,3,4,5]
for i in range(len(camino[0])):
  some_map.add_child(folium.Marker(location=[camino[3][i],camino[2][i]], popup=str(camino[0][i])))

some_map
  